In [1]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from smolagents import CodeAgent,DuckDuckGoSearchTool, HfApiModel,load_tool, tool, TransformersModel, Model, FinalAnswerTool, LiteLLMModel, UserInputTool
from custom_model import CustomTransformersModel
from typing import Optional
import torch
import yaml
import requests
from json import loads, dumps

In [2]:
from utils_tools import (get_price_from_product_url, search_on_amazon,
                         ParserProductDescriptionWithGuideTool, GetProductDescriptionTool,
                         CompareProductTool, FilterProduct, FinalAnswerTool)

In [3]:
from litellm import completion

In [4]:
with open('secrets.yaml') as f:
    SECRETS = yaml.safe_load(f)
os.environ["HF_TOKEN"] = SECRETS['hf_token']
os.environ["GEMINI_API_KEY"] = SECRETS['gemini_token']

In [5]:
model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')

In [6]:
final_answer = FinalAnswerTool()

In [6]:
compare_products = CompareProductTool(model)

In [20]:
tv_search_results = search_on_amazon(keyword="TV")
print(tv_search_results)

[{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)', 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg', 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6', 'price': '119,99 €', 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}, {'product_name': 'Smart Tech 24HN01V3 HD LED TV 24 Pouces (60cm) Triple Tuner Dolby Audio H.265

In [12]:
comparison_table = compare_products(tv_search_results)

In [13]:
comparison_table

,0
0,{'product_name': 'RCA TV 32 Pouces (80 cm) LED...
1,{'product_name': 'Smart Tech 24HN01V3 HD LED T...
2,{'product_name': '[App intégrée] Vidéoprojecte...
3,{'product_name': 'THOMSON 32'' HD LED Fire TV ...
4,{'product_name': 'TD Systems Smart TV 40 Pouce...


In [11]:
final_answer('blabla', comparison_table)

('blabla',
                                                    0
 0  {'product_name': 'RCA TV 32 Pouces (80 cm) LED...
 1  {'product_name': 'Smart Tech 24HN01V3 HD LED T...
 2  {'product_name': 'VidéoProjecteur Portable Min...
 3  {'product_name': 'THOMSON 32'' HD - LED Fire T...
 4  {'product_name': 'Samsung 55Q70D - TV QLED 55 ...)

In [27]:
product_description = tv_search_results[0]

In [28]:
product_description

{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)',
 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6',
 'price': '119,99 €',
 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}

In [21]:
model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')

In [29]:
messages = [{"role": "system",
                     "content": ("Tu es un super assistant très fort pour resumer et structurer des json."
                                 "Je vais te fournir un json et tu vas faire en sorte qu'aucune valeur de clef soit superieur à 50 characère."
                                 "Tu as le droit de resumer pour conserver que les informations principales mais tu nas pas le droit de toucher les champs qui sont des urls, tu les rends tel quel sans les affecter."
                                 "Ne change pas la structure du json, il ne doit pas manquer de clef qui étaient presentent initialement ni etre renommé"
                                 "concernant les champs de livraison, je veux que tu ne conserve que la date la plus courte de livraison sous le format jour mois")},
                    {"role": "user", "content": f"met moi en forme ce json stp : {product_description}"}]

In [33]:
res = model(messages, response_format={"type": "json"}).content

SyntaxError: invalid character '…' (U+2026) (3676893262.py, line 1)

In [31]:
product_description

{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)',
 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6',
 'price': '119,99 €',
 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}

In [39]:
res.replace('json', '', 1)

'```\n{\n  "product_name": "RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby",\n  "image_url": "https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg",\n  "product_link": "https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6",\n  "price": "119,99 €",\n  "delivery_date": "22 mai"\n}\n```'

In [ ]:
```json
{
  "product_name": "RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby",
  "image_url": "efeffef.jpg",
  "product_link": "feeffefe",
  "price": "119,99 €",
  "delivery_date": "22 mai"
}
```

In [37]:
chaine_json = json.dumps(res, indent=2, ensure_ascii=False)
print(chaine_json)

"```json\n{\n  \"product_name\": \"RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby\",\n  \"image_url\": \"https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg\",\n  \"product_link\": \"https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6\",\n  \"price\": \"119,99 €\",\n  \"delivery_date\": \"22 mai\"\n}\n```"


In [38]:
chaine_json

'"```json\\n{\\n  \\"product_name\\": \\"RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby\\",\\n  \\"image_url\\": \\"https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg\\",\\n  \\"product_link\\": \\"https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6\\",\\n  \\"price\\": \\"119,99 €\\",\\n  \\"delivery_date\\": \\"22 mai\\"\\n}\\n```"'

In [41]:
import json
json.loads(res.strip('```').replace('json', '', 1))

{'product_name': 'RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby',
 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6',
 'price': '119,99 €',
 'delivery_date': '22 mai'}

In [44]:
c = "blablabla json bblalajson et json"

In [42]:
res

'```json\n{\n  "product_name": "RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby",\n  "image_url": "https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg",\n  "product_link": "https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6",\n  "price": "119,99 €",\n  "delivery_date": "22 mai"\n}\n```'

In [7]:
computer_products = []                                                                                                                                                                                                                                                                                            
for keyword in ["computer", "laptop", "desktop"]:                                                                                                                                                                                                                                                                 
  products = search_on_amazon(keyword=keyword)                                                                                                                                                                                                                                                                  
  computer_products.extend(products)                                                                                                                                                                                                                                                                            
                                                                                                                                                                                                                                                                                                                
print(f"Found {len(computer_products)} products.")     

Found 15 products.


In [13]:
SYSTEM_PROMPT_GUIDED = """You are an expert assistant in product information extraction.

Based on a *product description* provided by the user, your job is to identify and extract the *requested attributes*, 
and organize them in a structured JSON format.

Your response must **always include at minimum** the following keys, even if they are not explicitly requested:
- "product_name"
- "image_url"
- "price"

For each requested attribute:
- If it is found in the description, provide its value.
- If it is missing, return `"N/A"` as the value.

Your final output must be a valid JSON object, using the exact attribute names as keys.

Example:
If the description is about a sofa and the requested attributes are ["dimension", "color", "material", "convertible"],
your output should look like this:

{
 "product_name": "Oslo 3-seater Sofa",
 "image_url": "https://...",
 "price": "€499",
 "dimension": "200x90x85 cm",
 "color": "Light grey",
 "material": "Fabric and wood",
 "convertible": "Yes"
}

Be precise, structured, and always do your best to help the customer understand the product clearly."""

In [21]:
get_product_description = GetProductDescriptionTool()
compare_products = CompareProductTool(model)
product_description_parser_with_guide = ParserProductDescriptionWithGuideTool(model, SYSTEM_PROMPT_GUIDED)

In [16]:
detailed_computer_products = []                                                                                                                                                                                                                                                                                   
features_to_extract = ["screen size", "processor", "RAM", "storage", "operating system"]                                                                                                                                                                                                                          
                                                                                                                                                                                                                                                                                                                
for product in computer_products:                                                                                                                                                                                                                                                                                 
  product_url = product["product_link"]                                                                                                                                                                                                                                                                         
  product_description = get_product_description(product_url)                                                                                                                                                                                                                                                    
  parsed_product = product_description_parser_with_guide(product_description, features_to_extract)                                                                                                                                                                                                               
  detailed_computer_products.append(parsed_product)                                                                                                                                                                                                                                                             
                                                                                                                                                                                                                                                                                                                
print(f"Parsed details for {len(detailed_computer_products)} products.")   

Parsed details for 15 products.


In [23]:
comparison_table = compare_products(detailed_computer_products)

JSONDecodeError: Extra data: line 12 column 1 (char 333)

In [46]:
import json

def _clean_product_info(product_description: dict):
    messages = [{"role": "system",
                 "content": ("Tu es un super assistant très fort pour resumer et structurer des json."
                             "Je vais te fournir un json et tu vas faire en sorte qu'aucune valeur de clef soit superieur à 50 characère."
                             "Tu as le droit de resumer pour conserver que les informations principales mais tu nas pas le droit de toucher les champs qui sont des urls, tu les rends tel quel sans les affecter."
                             "Ne change pas la structure du json, il ne doit pas manquer de clef qui étaient presentent initialement ni etre renommé"
                             "concernant les champs de livraison, je veux que tu ne conserve que la date la plus courte de livraison sous le format jour mois")},
                {"role": "user", "content": f"met moi en forme ce json stp : {product_description}"}]
    model_output = model(messages, response_format={"type": "json"}).content

    print(model_output)
    
    model_output = model_output.strip('```').replace('json', '', 1)
    
    print(model_output)
    return json.loads(model_output)

In [47]:
list_product_element = []

In [49]:
for product_index in range(len(detailed_computer_products)):
    list_product_element.append(_clean_product_info(detailed_computer_products[product_index]))

```json
{
  "product_name": "Ordinateur Portable 14,1\" Celeron Dual-Core",
  "image_url": "https://m.media-amazon.com/images/I/71lKWcPSzqL.__AC_SX300_SY300_QL70_ML2_.jpg",
  "price": "189,99€",
  "screen_size": "14.1 inches",
  "processor": "N4000 Celeron Dual-Core",
  "RAM": "6 Go",
  "storage": "128 Go de SSD",
  "operating system": "N/A"
}
```

{
  "product_name": "Ordinateur Portable 14,1\" Celeron Dual-Core",
  "image_url": "https://m.media-amazon.com/images/I/71lKWcPSzqL.__AC_SX300_SY300_QL70_ML2_.jpg",
  "price": "189,99€",
  "screen_size": "14.1 inches",
  "processor": "N4000 Celeron Dual-Core",
  "RAM": "6 Go",
  "storage": "128 Go de SSD",
  "operating system": "N/A"
}

```json
{
  "product_name": "PINSTONE PC Portable 16 Go RAM 512 Go SSD 15,6 Pouces",
  "image_url": "https://m.media-amazon.com/images/I/71E2qZw2xuL.__AC_SX300_SY300_QL70_ML2_.jpg",
  "price": "230,00€",
  "screen_size": "15,6 pouces",
  "processor": "Celeron N5095",
  "RAM": "16 Go",
  "storage": "512 Go SSD

JSONDecodeError: Extra data: line 12 column 1 (char 344)

In [39]:
product_index

6

In [43]:
t = """{
  "product_name": "PINSTONE PC Portable 16 Go RAM 512 Go SSD 15,6\"",
  "image_url": "https://m.media-amazon.com/images/I/71E2qZw2xuL.__AC_SX300_SY300_QL70_ML2_.jpg",
  "price": "230,00€",
  "screen size": "15,6 pouces",
  "processor": "Celeron N5095",
  "RAM": "16 Go",
  "storage": "512 Go SSD",
  "operating system": "N/A"
}
```"""

In [45]:
t.strip('```')

'{\n  "product_name": "PINSTONE PC Portable 16 Go RAM 512 Go SSD 15,6"",\n  "image_url": "https://m.media-amazon.com/images/I/71E2qZw2xuL.__AC_SX300_SY300_QL70_ML2_.jpg",\n  "price": "230,00€",\n  "screen size": "15,6 pouces",\n  "processor": "Celeron N5095",\n  "RAM": "16 Go",\n  "storage": "512 Go SSD",\n  "operating system": "N/A"\n}\n'